### Evaluate Topic Model in Python: Latent Dirichlet Allocation (LDA)

In the previous article, I introduced the concept of topic modeling and walked through the code for developing your first topic model using Latent Dirichlet Allocation (LDA) method in the python using sklearn implementation.

Pursuing on that understanding, in this article, I'll go a few steps deeper by outlining the framework to quantitatively evaluate topic models through the measure of topic coherence and share the code template in python using Gensim implementation to allow for end-to-end model development.

### Why evaluate topic models?

![img](https://tinyurl.com/y3xznjwq)

We know probabilistic topic models, such as LDA, are popular tools for analysis of the text, providing both a predictive and latent topic representation of the corpus. There is a longstanding assumption that the latent space discovered by these models is meaningful and useful, and evaluating such assumptions is challenging due to its unsupervised training process. There is a no-gold standard list of topics to compare against every corpus.

However, it is equally important to identify if a trained model is objectively good or bad, as well have an ability to compare different models/methods and to do so, we require an objective measure for the quality. Traditionally, and still for many practical applications, to evaluate if "the correct thing" has been learned about the corpus, we use implicit knowledge and "eyeballing" approaches. Ideally, we'd like to capture this information in a single metric that can be maximized, and compared. Let's take a look at roughly what approaches are commonly used for the evaluation:

**Eye Balling Models**
- Top N words
- Topics / Documents

**Intrinsic Evaluation Metrics**
- Capturing model semantics
- Topics interpretability

**Human Judgements**
- What is a topic

**Extrinsic Evaluation Metrics/Evaluation at task**
- Is model good at performing predefined tasks, such as classification

Natural language is messy, ambiguous and full of subjective interpretation, and sometimes trying to cleanse ambiguity reduces the language to an unnatural form. Nevertheless, in this article, we'll explore more about topic coherence, and how we can use it to quantitatively justify the model selection.

### What is Topic Coherence?

Perplexity is often used as an example of an intrinsic evaluation measure. It comes from the language modeling community and aims to capture how surprised a model is of new data it has not seen before. It is measured as the normalized log-likelihood of a held-out test set.

Focussing on the log-likelihood part, you can think of the perplexity metric as measuring how probable some new unseen data is given the model that was learned earlier. That is to say, how well does the model represent or reproduce the statistics of the held-out data.

However, past research has shown that predictive likelihood (or equivalently, perplexity) and human judgment are often not correlated, and even sometimes slightly anti-correlated. And that served as a motivation for more work trying to model the human judgment, and thus `Topic Coherence`.

The topic coherence concept combines a number of papers into one framework that allows evaluating the coherence of topics inferred by a topic model. But,

#### What is topic coherence?
Topic Coherence measures score a single topic by measuring the degree of semantic similarity between high scoring words in the topic. These measurements help distinguish between topics that are semantically interpretable topics and topics that are artifacts of statistical inference. But,

#### What is coherence?
A set of statements or facts is said to be coherent, if they support each other. Thus, a coherent fact set can be interpreted in a context that covers all or most of the facts. An example of a coherent fact set is "the game is a team sport", "the game is played with a ball", "the game demands great physical efforts"

### Coherence Measures

1. `C_v` measure is based on a sliding window, one-set segmentation of the top words and an indirect confirmation measure that uses normalized pointwise mutual information (NPMI) and the cosine similarity
2. `C_p` is based on a sliding window, one-preceding segmentation of the top words and the confirmation measure of Fitelson's coherence
3. `C_uci` measure is based on a sliding window and the pointwise mutual information (PMI) of all word pairs of the given top words
4. `C_umass` is based on document cooccurrence counts, a one-preceding segmentation and a logarithmic conditional probability as confirmation measure
5. `C_npmi` is an enhanced version of the C_uci coherence using the normalized pointwise mutual information (NPMI)
6. `C_a` is baseed on a context window, a pairwise comparison of the top words and an indirect confirmation measure that uses normalized pointwise mutual information (NPMI) and the cosine similarity

#### Install dependent packages

In [1]:
# To be run only once
if 0 == 1:
    !pip install gensim
    !pip install PyLDAvis
    !pip install spacy
    !python -m spacy download en_core_web_sm

    100% |████████████████████████████████| 1.6MB 759kB/s eta 0:00:01
    100% |████████████████████████████████| 296kB 4.1MB/s eta 0:00:01
    100% |████████████████████████████████| 163kB 5.4MB/s eta 0:00:01
    100% |████████████████████████████████| 235kB 4.5MB/s eta 0:00:01
    100% |████████████████████████████████| 92kB 11.9MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 12.0MB/s ta 0:00:01
  Running setup.py bdist_wheel for PyLDAvis ... done
  Stored in directory: /home/jupyter/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built PyLDAvis
    100% |████████████████████████████████| 10.2MB 121kB/s eta 0:00:01
    100% |████████████████████████████████| 2.1MB 632kB/s eta 0:00:01
    100% |████████████████████████████████| 122kB 10.4MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 9.7MB/s eta 0:00:01
    100% |████████████████████████████████| 3.7MB 353kB/s eta 0:00:01
    100% |██████████████████████████

### Model Implementation

#### Loading data

For this tutorial, we’ll use the dataset of papers published in NIPS conference. The NIPS conference (Neural Information Processing Systems) is one of the most prestigious yearly events in the machine learning community. The CSV data file contains information on the different NIPS papers that were published from 1987 until 2016 (29 years!). These papers discuss a wide variety of topics in machine learning, from neural networks to optimization methods, and many more.

<img src="https://s3.amazonaws.com/assets.datacamp.com/production/project_158/img/nips_logo.png" alt="The logo of NIPS (Neural Information Processing Systems)">

Let’s start by looking at the content of the file

In [1]:
# Importing modules
import pandas as pd
import os

os.chdir('..')

# Read data into papers
papers = pd.read_csv('./data/NIPS Papers/papers.csv')

# Print head
papers.head()

,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a..."


#### Data Cleaning

Since the goal of this analysis is to perform topic modeling, we will solely focus on the text data from each paper, and drop other metadata columns

In [2]:
# Remove the columns
papers = papers.drop(columns=['id', 'title', 'abstract', 
                              'event_type', 'pdf_name', 'year'], axis=1)

# sample only 100 papers
papers = papers.sample(100)

# Print out the first rows of papers
papers.head()

,paper_text
5614,Truncated Variance Reduction: A Unified Approa...
2870,"Stress, noradrenaline, and realistic predictio..."
4754,On a Theory of Nonparametric Pairwise Similari...
6515,An Analog Neural Network Inspired by\nFractal ...
5784,Hierarchical Deep Reinforcement Learning:\nInt...


#### Remove punctuation/lower casing

Next, let’s perform a simple preprocessing on the content of paper_text column to make them more amenable for analysis, and reliable results. To do that, we’ll use a regular expression to remove any punctuation, and then lowercase the text

In [3]:
# Load the regular expression library
import re

# Remove punctuation
papers['paper_text_processed'] = papers['paper_text'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
papers['paper_text_processed'] = papers['paper_text_processed'].map(lambda x: x.lower())

# Print out the first rows of papers
papers['paper_text_processed'].head()

5614    truncated variance reduction: a unified approa...
2870    stress noradrenaline and realistic prediction ...
4754    on a theory of nonparametric pairwise similari...
6515    an analog neural network inspired by\nfractal ...
5784    hierarchical deep reinforcement learning:\nint...
Name: paper_text_processed, dtype: object

#### Tokenize words and further clean-up text

Let’s tokenize each sentence into a list of words, removing punctuations and unnecessary characters altogether.

In [4]:
import gensim
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = papers.paper_text_processed.values.tolist()
data_words = list(sent_to_words(data))

print(data_words[:1][0][:30])

[u'truncated', u'variance', u'reduction', u'unified', u'approach', u'to', u'bayesian', u'optimization', u'and', u'level', u'set', u'estimation', u'ilija', u'bogunovic', u'jonathan', u'scarlett', u'andreas', u'krause', u'volkan', u'cevher', u'laboratory', u'for', u'information', u'and', u'inference', u'systems', u'lions', u'epfl', u'learning', u'and']


#### Creating Bigram and Trigram Models

Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring. Some examples in our example are: 'back_bumper', 'oil_leakage', 'maryland_college_park' etc.

Gensim's Phrases model can build and implement the bigrams, trigrams, quadgrams and more. The two important arguments to Phrases are min_count and threshold.

In [5]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

#### Remove Stopwords, Make Bigrams and Lemmatize

The phrase models are ready. Let’s define the functions to remove the stopwords, make trigrams and lemmatization and call them sequentially.

In [6]:
# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

Let's call the functions in order.

In [8]:
import spacy

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

[u'truncate', u'variance_reduction', u'unify', u'approach', u'bayesian', u'optimization', u'level', u'set', u'estimation', u'system', u'lion', u'epfl', u'learn', u'system', u'present', u'new', u'truncate', u'treat', u'bayesian', u'optimization', u'level', u'set', u'estimation', u'lse', u'gaussian_processe', u'unify', u'fashion', u'greedily', u'shrink', u'sum']


### Data transformation: Corpus and Dictionary

The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus. Let’s create them.

In [9]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 10), (3, 7), (4, 1), (5, 3), (6, 1), (7, 3), (8, 1), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 4), (15, 1), (16, 1), (17, 2), (18, 1), (19, 13), (20, 3), (21, 1), (22, 1), (23, 1), (24, 7), (25, 3), (26, 1), (27, 3), (28, 4), (29, 3)]


### Building the base topic model

We have everything required to train the base LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well. Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior (we'll use default for the base model).

chunksize controls how many documents are processed at a time in the training algorithm. Increasing chunksize will speed up training, at least as long as the chunk of documents easily fit into memory.

passes controls how often we train the model on the entire corpus (set to 10). Another word for passes might be "epochs". iterations is somewhat technical, but essentially it controls how often we repeat a particular loop over each document. It is important to set the number of "passes" and "iterations" high enough.

In [10]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

#### View the topics in LDA model
The above LDA model is built with 10 different topics where each topic is a combination of keywords and each keyword contributes a certain weightage to the topic.

You can see the keywords for each topic and the weightage(importance) of each keyword using `lda_model.print_topics()`

In [11]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  u'0.010*"target" + 0.010*"neuron" + 0.009*"network" + 0.008*"time" + 0.007*"model" + 0.007*"protein" + 0.007*"search" + 0.007*"image" + 0.007*"current" + 0.006*"error"'),
 (1,
  u'0.016*"model" + 0.011*"use" + 0.009*"network" + 0.007*"time" + 0.007*"learn" + 0.007*"training" + 0.006*"datum" + 0.006*"set" + 0.006*"function" + 0.005*"system"'),
 (2,
  u'0.013*"network" + 0.010*"object" + 0.008*"point" + 0.008*"problem" + 0.008*"use" + 0.008*"value" + 0.007*"formulation" + 0.007*"base" + 0.007*"view" + 0.007*"figure"'),
 (3,
  u'0.015*"model" + 0.012*"datum" + 0.011*"set" + 0.010*"use" + 0.008*"image" + 0.008*"learn" + 0.006*"matrix" + 0.006*"problem" + 0.006*"feature" + 0.006*"give"'),
 (4,
  u'0.022*"network" + 0.013*"training" + 0.012*"set" + 0.010*"color" + 0.010*"performance" + 0.009*"datum" + 0.009*"committee" + 0.009*"arm" + 0.008*"use" + 0.008*"sample"'),
 (5,
  u'0.012*"model" + 0.008*"visual" + 0.007*"network" + 0.007*"function" + 0.006*"problem" + 0.006*"datum" + 0.005*"

#### Compute Model Perplexity and Coherence Score

Let's calculate the baseline coherence score

In [12]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

('Coherence Score: ', 0.26431517844971164)


### Hyperparameter tuning

First, let's differentiate between model hyperparameters and model parameters :

- `Model hyperparameters` can be thought of as settings for a machine learning algorithm that are tuned by the data scientist before training. Examples would be the number of trees in the random forest, or in our case, number of topics K

- `Model parameters` can be thought of as what the model learns during training, such as the weights for each word in a given topic.

Now that we have the baseline coherence score for the default LDA model, let's perform a series of sensitivity tests to help determine the following model hyperparameters: 
- Number of Topics (K)
- Dirichlet hyperparameter alpha: Document-Topic Density
- Dirichlet hyperparameter beta: Word-Topic Density

We'll perform these tests in sequence, one parameter at a time by keeping others constant and run them over the two difference validation corpus sets. We'll use `C_v` as our choice of metric for performance comparison 

In [13]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

Let's call the function, and iterate it over the range of topics, alpha, and beta parameter values

In [14]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.75), 
               corpus]

corpus_title = ['100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

100%|██████████| 270/270 [47:36<00:00, 10.58s/it]


### Final Model Training

Based on external evaluation (Code to be added from Excel based analysis), train the final model

In [15]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.01,
                                           eta=0.9)

In [16]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  u'0.003*"cell" + 0.003*"current" + 0.002*"equipartition" + 0.002*"unit" + 0.002*"source_separation" + 0.002*"density" + 0.002*"digital" + 0.002*"chip" + 0.002*"som" + 0.001*"source"'),
 (1,
  u'0.016*"network" + 0.008*"model" + 0.008*"use" + 0.007*"training" + 0.006*"learn" + 0.006*"system" + 0.006*"weight" + 0.006*"neural" + 0.006*"state" + 0.005*"train"'),
 (2,
  u'0.008*"object" + 0.005*"view" + 0.005*"formulation" + 0.004*"permutation" + 0.003*"gain" + 0.003*"permutahedron" + 0.002*"system" + 0.002*"model" + 0.002*"sorting_network" + 0.002*"current"'),
 (3,
  u'0.008*"datum" + 0.006*"set" + 0.005*"model" + 0.005*"matrix" + 0.004*"number" + 0.004*"use" + 0.004*"cluster" + 0.004*"rank" + 0.003*"perturbation" + 0.003*"dimension"'),
 (4,
  u'0.000*"color" + 0.000*"set" + 0.000*"use" + 0.000*"network" + 0.000*"model" + 0.000*"image" + 0.000*"function" + 0.000*"training" + 0.000*"time" + 0.000*"point"'),
 (5,
  u'0.009*"model" + 0.007*"visual" + 0.006*"neuron" + 0.005*"network" + 

In [17]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

LDAvis_prepared

/home/jupyter/.local/lib/python2.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
7      58.895950        1       1 -0.168808 -0.034076
1      17.232506        1       2 -0.076027  0.055721
5      12.918425        1       3 -0.053432  0.054010
6       6.068927        1       4 -0.022521 -0.047228
3       3.192707        1       5  0.023060 -0.043905
2       1.241229        1       6  0.070890  0.008933
0       0.449649        1       7  0.107736  0.008278
4       0.000600        1       8  0.119102 -0.001734, topic_info=     Category         Freq         Term        Total  loglift  logprob
2368  Default   859.000000      network   859.000000  30.0000  30.0000
2732  Default  1055.000000        datum  1055.000000  29.0000  29.0000
1366  Default  1229.000000          set  1229.000000  28.0000  28.0000
7552  Default  1387.000000        model  1387.000000  27.0000  27.0000
1441  Default   609.000000        error   609.000000  26.0000  26.0000
7713  Default   390.000000       matrix   390.000000  25.0000  25.0000
8298  Default  1311.000000          use  1311.000000  24.0000  24.0000
249   Default   164.000000       object   164.000000  23.0000  23.0000
5582  Default   320.000000      cluster   320.000000  22.0000  22.0000
1832  Default   409.000000       system   409.000000  21.0000  21.0000
1274  Default   560.000000       number   560.000000  20.0000  20.0000
7303  Default   292.000000         term   292.000000  19.0000  19.0000
7255  Default   609.000000       figure   609.000000  18.0000  18.0000
1482  Default   409.000000        input   409.000000  17.0000  17.0000
4225  Default   266.000000       neuron   266.000000  16.0000  16.0000
569   Default   122.000000      current   122.000000  15.0000  15.0000
4364  Default   328.000000       weight   328.000000  14.0000  14.0000
4166  Default   456.000000        state   456.000000  13.0000  13.0000
716   Default   612.000000     training   612.000000  12.0000  12.0000
615   Default   185.000000       visual   185.000000  11.0000  11.0000
6438  Default   101.000000  formulation   101.000000  10.0000  10.0000
1004  Default   456.000000        image   456.000000   9.0000   9.0000
522   Default   667.000000         time   667.000000   8.0000   8.0000
4515  Default   614.000000         give   614.000000   7.0000   7.0000
6463  Default   380.000000       neural   380.000000   6.0000   6.0000
8523  Default   106.000000         view   106.000000   5.0000   5.0000
310   Default   198.000000       convex   198.000000   4.0000   4.0000
7328  Default   212.000000       theory   212.000000   3.0000   3.0000
5232  Default   731.000000         show   731.000000   2.0000   2.0000
4434  Default   422.000000       vector   422.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
3335   Topic8     0.000118     darkness     4.775212   1.4186  -9.0430
1825   Topic8     0.000152        scene    40.272274  -0.4652  -8.7945
3765   Topic8     0.000138     subtract    16.983007   0.3013  -8.8915
4700   Topic8     0.000124         blur     7.258564   1.0485  -8.9944
4122   Topic8     0.000124       indoor     7.260155   1.0479  -8.9947
7552   Topic8     0.000221        model  1387.270386  -3.6298  -8.4197
716    Topic8     0.000194     training   612.136902  -2.9397  -8.5478
809    Topic8     0.000128       bright     9.546785   0.8027  -8.9661
2132   Topic8     0.000201     function   912.380981  -3.3025  -8.5115
522    Topic8     0.000194         time   667.092590  -3.0262  -8.5483
5836   Topic8     0.000190        point   646.042480  -3.0148  -8.5690
1274   Topic8     0.000186       number   560.526489  -2.8922  -8.5883
3479   Topic8     0.000188         base   638.654480  -3.0153  -8.5809
4515   Topic8     0.000184         give   614.091980  -2.9968  -8.6017
4180   Topic8     0.000189      problem   899.662292  -3.3487  -8.5717
3329   Topic8     0.000181        value   597.269104  -2.9850  -8.6176
5210   To